In [1]:
!ls

ERROR: Error in !ls: invalid argument type


In [2]:
library("rjags")
mu.frac <- c(0.074, 0.32)
gamma.mean.dn <- c(20, 4.7)
beta.dn <- c(1,1)
gamma.mean.CC <- c(2.3, 1.00)
beta.CC <- c(4.0, 1000)
rho1 <- c(0.1, 0.5)
nu1 <- c(200, 100)
rho0 <- c(0.1, 0.5)
nu0 <- c(200, 100)
hyperpar <- as.array(rbind(gamma.mean.dn, beta.dn, gamma.mean.CC, beta.CC, rho1, nu1, rho0, nu0))
l <- 100
pi0 <- 0.94 # the fraction of non-risk genes

# ASC (Autism Sequencing Consortium) data
# The file name contains the sample size information
# The only relevant counts are dn.LoF and dn.mis3
data <- read.csv("TADA/data/ASC_2231trios_1333trans_1601cases_5397controls.csv", header=TRUE, as.is=TRUE)
ntrio <- 2231  # number of trios
ncase <- 1601  # number of cases
nctrl <- 5397  # number of controls
ntrans <- 1333 # number of subjects with transmission data
N <- list(dn=ntrio, ca=ntrans+ncase, cn=ntrans+nctrl)

# Running TADA
counts <- as.array(cbind(data$dn.LoF, data$case.LoF+data$trans.LoF, data$ctrl.LoF+data$ntrans.LoF, data$dn.mis3, data$case.mis3+data$trans.mis3,
 data$ctrl.mis3+data$ntrans.mis3))


count.mu.Data <- cbind(counts[, 1:3], data$mut.rate*mu.frac[1])

Loading required package: coda
Linked to JAGS 3.4.0
Loaded modules: basemod,bugs


In [3]:
y <- as.numeric(counts[, 1])
murate = data$mut.rate*mu.frac[1]

Ncount <- N
NN <- length(y)

dataJags <- list(y = y,
              alpha0 = c(1, 1),
              NN = NN,
              T = c(NA, rep(NA, NN - 2), NA),
              murate = murate,
#              m1 = m1, m2 = m2,
              Ndn = N$dn)

jagsFile = "model {

           p1[1:2] ~ ddirch(alpha0[])

           gamma.mean ~ dunif(5, 30)
           beta.mean ~ dunif(0.5, 5)

for (i in 1:NN){
           gammaAll[i] ~ dgamma(gamma.mean*beta.mean, beta.mean)

           m[i, 1] <- 2*Ndn*murate[i]
           m[i, 2] <- 2*Ndn*murate[i]*gammaAll[i]


           y[i] ~ dpois(mu0[i])
           mu0[i] <- m[i, T[i]]
           T[i] ~ dcat(p1[])
       }

}
"

In [4]:
n.chains = 1
n.adapt = 100
        mixture <- jags.model(textConnection(jagsFile),# inits = inits,
               data = dataJags, n.chains = n.chains, n.adapt = n.adapt)



Compiling model graph
   Resolving undeclared variables
   Allocating nodes
   Graph Size: 116575

Initializing model



In [5]:
nUpdate = 2000
update(mixture, nUpdate)


In [ ]:
n.iter = 4000
thin = 2
xem <- coda.samples(mixture, c('mu0', 'T', 'gammaAll', 'gamma.mean', 'beta.mean', 'p1'), 
                    n.iter = n.iter, thin = thin)
##Each chain will produce a list



In [116]:
##Calculate T
T.sample <- apply(a1[, 1:NN], 2, function(xTT) {
    kk <- table(xTT)
    kk1 <- as.numeric(names(which(kk == max(kk))))
    kk2 <- ifelse(length(kk1) > 1, 2, kk1)
    return(kk2)

})
T.sample <- unlist(T.sample)

table(T.sample)
table(T.sample)/NN

b11 <- cbind(data[, 1], T.sample)
write.table(b11, "list.out.txt", col.names=FALSE, row.names=FALSE, quote = FALSE)

apply(a1[, 56208:56209], 2, mean)


Warning message:
In jags.model(textConnection(jagsFile), inits = inits, data = dataJags, : Unused variable "nLength" in dataWarning message:
In jags.model(textConnection(jagsFile), inits = inits, data = dataJags, : Unused variable "gamma.mean" in dataWarning message:
In jags.model(textConnection(jagsFile), inits = inits, data = dataJags, : Unused variable "beta.dn" in data

Compiling model graph
   Resolving undeclared variables
Deleting model



ERROR: Error in jags.model(textConnection(jagsFile), inits = inits, data = dataJags, : RUNTIME ERROR:
Index out of range for node T




Warning message:
In jags.model(textConnection(jagsFile), data = dataJags, n.chains = 1, : Unused variable "nGroups" in dataWarning message:
In jags.model(textConnection(jagsFile), data = dataJags, n.chains = 1, : Unused variable "alpha0" in dataWarning message:
In jags.model(textConnection(jagsFile), data = dataJags, n.chains = 1, : Unused variable "T" in data

Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Deleting model



ERROR: Error in jags.model(textConnection(jagsFile), data = dataJags, n.chains = 1, : RUNTIME ERROR:
Compilation error on line 22.
xData[1] is a logical node and cannot be observed




Compiling model graph
   Resolving undeclared variables
   Allocating nodes
   Graph Size: 404

Initializing model



In [60]:
xem

[[1]]
Markov Chain Monte Carlo (MCMC) output:
Start = 1001 
End = 2000 
Thinning interval = 1 
        group[1] group[2] group[3] group[4] group[5] group[6] group[7] group[8]
   [1,]        1        1        0        1        1        0        0        1
   [2,]        1        1        1        1        0        0        0        1
   [3,]        1        1        1        1        0        0        1        1
   [4,]        1        1        1        1        1        0        0        1
   [5,]        1        1        1        1        1        0        0        1
   [6,]        1        1        1        1        1        0        0        1
   [7,]        1        1        1        1        1        0        0        1
   [8,]        1        1        1        1        1        0        0        1
   [9,]        1        1        0        1        1        0        0        1
  [10,]        1        1        1        1        0        0        0        1
  [11,]        1        1

In [59]:
 allResults <- t(apply(xem[[1]], 2, function(x)
                    bb <- c(mean(x), quantile(x, c(0.25, 0.75)))))
allResults

,,25%,75%
group[1],0.999,1,1
group[2],0.995,1,1
group[3],0.879,1,1
group[4],0.985,1,1
group[5],0.767,1,1
group[6],0.069,0,0
group[7],0.165,0,0
group[8],0.972,1,1
group[9],0.943,1,1
group[10],0.981,1,1


In [ ]:
model {         
    for(j in 1:N) {
        t.1[j] ~ dweib(alpha.1,lambda.1) 
        t.2[j] ~ dweib(alpha.2, lambda.2);
        p.det[j] ~ dbeta(a,b);
        y[j] <- step(t.2[j]-t.1[j]);            
        y[j] ~ dbern(py);
    }   
    alpha.1 ~ dgamma(0.3,0.0001); 
    lambda.1 ~ dnorm(0., 10000.);
    alpha.2 ~ dgamma(0.3,0.0001); 
    lambda.2 ~ dnorm(0., 10000.);   
    py ~ dbeta(0.3,0.3);
    a ~ dgamma(1, 0.01) 
    b ~ dgamma(1, 0.01) 
    pd ~ dbeta(a,b) 
    p.f<- pd*py
}

data {
  for (j in 1:N) {
    y[j] <- step(t.2[j] - t.1[j]) 
  }
}